## This jupyter notebook will take a directory full of raw ADNI data and preprocess it in-place.

### In particular, this script: 
### $\bullet$ Uses MRIcron to convert the .dcm filed into .nii.gz files. (Google for installation help)
### $\bullet$ Uses FSL to affine register and skull strip the samples. (See fsl_anat on FSL's website)

In [1]:
import numpy as np
import scipy.io as sio
import os
import pydicom
from subprocess import call
import re

In [2]:
regex = re.compile('(.*gz$)')
brain = re.compile('(brain.*nii.*gz$)')

In [3]:
patient_condition = 'AD' # set this to 'NC' to traverse the other folder

adni_file_path = "" + os.getcwd() + "/ADNI"
print(adni_file_path)

listFilesDCM = []
folders_level1 = os.listdir(adni_file_path)

numSamples = len(folders_level1)
print(numSamples)

/home/parallels/Alz-Finders/ADNI
66


In [4]:
counter = 0
for sample_idx in range(len(folders_level1)):
    sample = folders_level1[sample_idx]
    sample_file_path = os.path.join(adni_file_path, sample)
    root_sample = sample_file_path
    sample_file_path = os.path.join(sample_file_path, "MPRAGE")
    ref_sample_file_path = sample_file_path
    
    for jj in range(len(os.listdir(sample_file_path))):
        sample_file_path = os.path.join(ref_sample_file_path, os.listdir(ref_sample_file_path)[jj])      
        
        for ii in range(len(os.listdir(sample_file_path))):
            sample_file_path = os.path.join(sample_file_path, os.listdir(sample_file_path)[ii])
            
            #Check if there is already a .nii file. If so, skip this folder.
            breakout = False
            for nii in os.listdir(sample_file_path):
                if regex.match(nii):
                    breakout = True
            if breakout:
                pass
                counter += 1
                break
            
            #Run the conversion tool for this directory of dmc files.
            #call(["dcm2nii", "-v", "y", "-o", root_sample, sample_file_path])
            call(["dcm2nii", "-v", "y", sample_file_path])
            
            print(str(counter) + " " + str(sample_file_path))
            counter += 1
            

    

OSError: [Errno 20] Not a directory: '/home/parallels/Alz-Finders/ADNI/003_S_0931/MPRAGE/2006-10-11_11_49_21.0.anat/T1_orig.nii.gz'

In [5]:
counter = 0
already = 0
unproc = 0
cont = 0

for sample_idx in range(len(folders_level1)):
    sample = folders_level1[sample_idx]
    sample_file_path = os.path.join(adni_file_path, sample)
    root_sample = sample_file_path
    
   
    sample_file_path = os.path.join(sample_file_path, "MPRAGE")
    #print(os.listdir(sample_file_path))
    ref_sample_file_path = sample_file_path
    
    for jj in range(len(os.listdir(sample_file_path))):
        sample_file_path = os.path.join(ref_sample_file_path, os.listdir(ref_sample_file_path)[jj])
        #print("1 " + str(sample_file_path))
        #If this folder has already been preprocessed
        if (os.listdir(ref_sample_file_path)[jj] + ".anat") in os.listdir(ref_sample_file_path):
            #print(str(os.listdir(ref_sample_file_path)[jj]) + "   Already Processed ")
            already +=1
            print(str(already) + " already")
            
            continue
            
        if (sample_file_path[-1]) == "t":
                #print(sample_file_path + "  SKIPPED")
                #counter += 1
                #print(str(counter) + " counter")
                #print(sample_file_path)
                continue
        
            
        for ii in range(len(os.listdir(sample_file_path))):
            
            out = (sample_file_path)  
            #print(sample_file_path)
            #print(ii)
            sample_file_path = os.path.join(sample_file_path, os.listdir(sample_file_path)[ii])           
            breakout = False
            #Don't try to preproccess the output folders
            if (sample_file_path[-1]) == "z":
                counter += 1
                continue
                
            #print(sample_file_path)
            
            for nii in os.listdir(sample_file_path):
                #print(nii)
                if (nii[0] == "c") or (nii[0] == 'o'):
                    continue
                    
                if regex.match(nii):
                    breakout = True
                    #print(nii)
                    In = os.path.join(sample_file_path,nii)
                    #print("UNPROCESSED : " + str(unproc) + " " + str(In) + " ")
                    unproc += 1
                    print("Processing " + str(In) )
                    print(cont)
                    cont += 1
                    print(nii)
                    call(["fsl_anat", "--noseg", "--nosubcortseg", "--nobias", "--nocrop", "--clobber", "-o", out, "-i", In])
                    #print(":) " + str(counter) + "\n")
                    #counter += 1
                    
            if breakout:
                #counter += 1
                break         
            
            print("\n\n\n\n\n NO .nii FILE FOUND \n\n\n\n\n")
            #counter += 1
            
            break


1 already
2 already
3 already
4 already
5 already
6 already
7 already
8 already
9 already
10 already
11 already
12 already
13 already
14 already
15 already
16 already
17 already
18 already
19 already
20 already
21 already
22 already
23 already
24 already
25 already
26 already
27 already
28 already
29 already
30 already
31 already
32 already
33 already
34 already
35 already
36 already
37 already
38 already
39 already
40 already
41 already
42 already
43 already
44 already
45 already
46 already
47 already
48 already
49 already
50 already
51 already
52 already
53 already
54 already
55 already
56 already
57 already
58 already
59 already
60 already
61 already
62 already
63 already
64 already
65 already
66 already
67 already
68 already
69 already
70 already
71 already
72 already
73 already
74 already
75 already
76 already
77 already
78 already
79 already
80 already
81 already
82 already
83 already
84 already
85 already
86 already
87 already
88 already
89 already
90 already
91 already
92 alrea

# This next block erases all uneeded raw data, beware. 

In [ ]:
counter = 0
unproc = 0
cont = 0

adni_file_path = "" + os.getcwd() + "/ADNI"
groups = ["AD", "CN"]

ref = "/home/parallels/Alz-Finders/REF.nii.gz"

visited = set()



for group in groups:
    
    output_base = "/home/parallels/Alz-Finders/ADNI/" + group
    
    already = 0
    adni_file_path = "" + os.getcwd() + "/ADNI_"+group
    folders_level1 = os.listdir(adni_file_path)
    numSamples = len(folders_level1)
    
    
    for sample_idx in range(len(folders_level1)):
        sample = folders_level1[sample_idx]
        sample_file_path = os.path.join(adni_file_path, sample)
        root_sample = sample_file_path
        

        sample_file_path = os.path.join(sample_file_path, "MPRAGE")
        #print(os.listdir(sample_file_path))
        ref_sample_file_path = sample_file_path

        for jj in range(len(os.listdir(sample_file_path))):
            
            sample_file_path = os.path.join(ref_sample_file_path, os.listdir(ref_sample_file_path)[jj])
            date = os.listdir(ref_sample_file_path)[jj]
            #print("1 " + str(sample_file_path))
            #If this folder has already been preprocessed
            if (os.listdir(ref_sample_file_path)[jj] + ".anat") in os.listdir(ref_sample_file_path):
                #print(str(os.listdir(ref_sample_file_path)[jj]) + "   Already Processed ")
                #print(str(os.listdir(ref_sample_file_path)[jj]) + ".anat")
                #print(sample_file_path + ".anat/T1_biascorr_brain.nii.gz")
                #print(output_base + "/" + sample + "/" + date + ".nii.gz")
                
                if sample not in os.listdir(output_base):
                    call(["mkdir", output_base + "/" + sample])
                
                print( reference )
                
                In = sample_file_path + ".anat/T1_biascorr_brain.nii.gz"
                Out = output_base + "/" + sample + "/" + date + ".nii.gz"
                
                #call(["cp", sample_file_path + ".anat/T1_biascorr_brain.nii.gz", output_base + "/" + sample + "/" + date + ".nii.gz"])
                call(["flirt", "-ref", ref, "-in", In, "-out", Out])
                
                already +=1
                #print(str(already) + " already")
                print(In)
                call(["flirt", "-ref", ref, "-in", In, "-out", Out])
                #print(Out)
                print("\n")

                continue

            if (sample_file_path[-1]) == "t":
                    #print(sample_file_path + "  SKIPPED")
                    #counter += 1
                    #print(str(counter) + " counter")
                    #print(sample_file_path)
                    continue


            for ii in range(len(os.listdir(sample_file_path))):

                out = (sample_file_path)  
                #print(sample_file_path)
                #print(ii)
                sample_file_path = os.path.join(sample_file_path, os.listdir(sample_file_path)[ii])           
                breakout = False
                #Don't try to preproccess the output folders
                if (sample_file_path[-1]) == "z":
                    counter += 1
                    continue

                


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/023_S_0139/MPRAGE/2007-02-09_09_55_08.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/023_S_0139/MPRAGE/2007-02-09_09_34_44.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/023_S_0139/MPRAGE/2008-03-04_07_32_59.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/023_S_0139/MPRAGE/2006-01-24_11_12_57.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/012_S_0689/MPRAGE/2007-07-19_10_48_16.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/012_S_0689/MPRAGE/2007-07-19_10_57_26.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/013_S_1205/MPRAGE/2007-01-11_13_55_08.0.ana



/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/012_S_0720/MPRAGE/2007-08-21_10_35_07.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/012_S_0720/MPRAGE/2007-08-21_10_43_51.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/020_S_0213/MPRAGE/2006-03-22_13_33_31.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/020_S_0213/MPRAGE/2007-06-04_13_21_27.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/013_S_0699/MPRAGE/2006-08-28_15_00_26.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/036_S_0759/MPRAGE/2008-09-10_13_26_02.0.anat/T1_biascorr_brain.nii.gz


/home/parallels/Alz-Finders/REF.nii.gz
/home/parallels/Alz-Finders/ADNI_AD/036_S_0759/MPRAGE/2007-08-30_12_08_53.0.a